In [ ]:
import glob
import os
import sys
import sys
import json
import math
import datetime
import tensorflow as tf
import random
from tensorflow.keras import layers, Model
from tqdm import tqdm
assert tf.version.VERSION >= "2.4.0", "version of tf must greater/equal than 2.4.0"
#print(tf.__path__)
print(tf.version.VERSION)
if not os.path.exists("./save_weights"):
    os.makedirs("./save_weights")

In [ ]:
im_height = 224
im_width = 224
batch_size = 16
epochs = 30
num_classes = 5

log_dir = "./logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_writer = tf.summary.create_file_writer(os.path.join(log_dir, "train"))
val_writer = tf.summary.create_file_writer(os.path.join(log_dir, "val"))

#data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
#image_path = os.path.join(data_root, "kaggle/input/flowerdata", "flower_data")
image_path = './flower_data'
assert os.path.exists(image_path),"file:'{}' does not exist".format(image_path)


In [ ]:
train_dir = os.path.join(image_path, "train")
val_dir = os.path.join(image_path, "val")

random.seed(0)
# class dict
data_class = [cla for cla in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, cla))]
class_num = len(data_class)
class_dict = dict((value, index) for index, value in enumerate(data_class))


train_image_path = glob.glob(train_dir + "/*/*.jpg")
random.shuffle(train_image_path)
train_num = len(train_image_path)
train_label_list = [class_dict[path.split(os.path.sep)[-2]] for path in train_image_path]
print(train_label_list[:1])


val_image_path = glob.glob(val_dir + "/*/*.jpg")
random.shuffle(val_image_path)
val_num = len(val_image_path)
val_label_list = [class_dict[path.split(os.path.sep)[-2]] for path in val_image_path]

print("using {} images for training, {} images for validation.".format(train_num, val_num))


In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
def process_train_img(img_path, label):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels= 3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_crop_or_pad(image, im_height, im_width)
    image = tf.image.random_flip_left_right(image)
    image = (image - mean) / std
    return image, label

def process_val_img(img_path, label):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_crop_or_pad(image, im_height, im_width)
    image = (image - mean) / std
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_path,train_label_list))
total_train = len(train_image_path)
train_dataset = train_dataset.cache().map(process_train_img, num_parallel_calls=AUTOTUNE)\
                             .shuffle(buffer_size= total_train)\
                             .batch(batch_size)\
                             .prefetch(buffer_size=AUTOTUNE)


val_dataset = tf.data.Dataset.from_tensor_slices((val_image_path,val_label_list))
total_val = len(val_image_path)
val_dataset = val_dataset.cache().map(process_train_img, num_parallel_calls=AUTOTUNE)\
                         .batch(batch_size)\
                         .prefetch(buffer_size=AUTOTUNE)


In [ ]:
class ConvBNReLU(layers.Layer):
    def __init__(self, filters = 1, kernel_size = 1, strides = 1, padding = 'same', **kwargs):
        super(ConvBNReLU, self).__init__(**kwargs)

        self.conv = layers.Conv2D(filters=filters,
                                  kernel_size=kernel_size,
                                  strides=strides,
                                  padding=padding,
                                  use_bias=False,
                                  kernel_regularizer=tf.keras.regularizers.l2(4e-5),
                                  name="conv1")
        self.bn = layers.BatchNormalization(momentum=0.9, name="bn")
        self.relu = layers.ReLU()

    def call(self, inputs, training=None, **kwargs):
        x = self.conv(inputs)
        x = self.bn(x, training=training)
        x = self.relu(x)
        return x

class DWConvBN(layers.Layer):
    def __init__(self, kernel_size = 3, strides = 1, padding = 'same', **kwargs):
        super(DWConvBN, self).__init__(**kwargs)
        self.dw_conv = layers.DepthwiseConv2D(kernel_size=kernel_size,
                                              strides=strides,
                                              padding=padding,
                                              use_bias=False,
                                              kernel_regularizer=tf.keras.regularizers.l2(4e-5),
                                              name="dw1")

        self.bn = layers.BatchNormalization(momentum=0.9, name="bn")

    def call(self, inputs, training=None, **kwargs):
        x = self.dw_conv(inputs)
        x = self.bn(x, training=training)
        return x

class ChannelShuffle(layers.Layer):
    def __init__(self, shape, groups = 2, **kwargs):
        super(ChannelShuffle, self).__init__(**kwargs)
        batch_size, height, width, num_channels = shape
        assert num_channels % 2 == 0
        channel_per_group = num_channels // groups

        self.reshape1 = layers.Reshape((height, width, groups, channel_per_group))
        self.reshape2 = layers.Reshape((height, width, num_channels))

    def call(self, inputs, **kwargs):
        x = self.reshape1(inputs)
        x = tf.transpose(x, perm=[0, 1, 2, 4, 3])
        x = self.reshape2(x)
        return x


class ChannelSplit(layers.Layer):
    def __init__(self, num_splits = 2, **kwargs):
        super(ChannelSplit, self).__init__(**kwargs)
        self.num_splits = num_splits

    def call(self, inputs, **kwargs):
        b1, b2 = tf.split(inputs, num_or_size_splits=self.num_splits, axis=-1)
        return b1, b2


def shuffle_block_s1(inputs, output_c, stride, prefix):
    if stride != 1:
        raise ValueError("illegal stride value.")

    assert output_c % 2 == 0
    branch_c = output_c // 2

    print("inputs:{} output_c:{} stride:{} prefix:{}".format(inputs, output_c, stride, prefix))
    x1, x2 = ChannelSplit(name=prefix + "/split")(inputs)
    print(x1)

    x2 = ConvBNReLU(filters = branch_c, name = prefix + "/b2_conv1")(x2)
    x2 = DWConvBN(kernel_size = 3, strides = stride, name = prefix + "/b2_dw1")(x2)
    x2 = ConvBNReLU(filters=branch_c, name = prefix + "/b2_conv2")(x2)

    x = layers.Concatenate(name=prefix + "/concat")([x1, x2])
    x = ChannelShuffle(x.shape, name = prefix + "/channelshuffle")(x)

    return x


def shuffle_block_s2(inputs, output_c: int, stride: int, prefix: str):
    if stride != 2:
        raise ValueError("illegal stride value.")

    assert output_c % 2 == 0
    branch_c = output_c // 2

    # shortcut branch
    x1 = DWConvBN(kernel_size=3, strides=stride, name=prefix + "/b1_dw1")(inputs)
    x1 = ConvBNReLU(filters=branch_c, name=prefix + "/b1_conv1")(x1)

    # main branch
    x2 = ConvBNReLU(filters=branch_c, name=prefix + "/b2_conv1")(inputs)
    x2 = DWConvBN(kernel_size=3, strides=stride, name=prefix + "/b2_dw1")(x2)
    x2 = ConvBNReLU(filters=branch_c, name=prefix + "/b2_conv2")(x2)

    x = layers.Concatenate(name=prefix + "/concat")([x1, x2])
    x = ChannelShuffle(x.shape, name=prefix + "/channelshuffle")(x)

    return x


def shufflenet_v2(num_classes: int,
                  input_shape: tuple,
                  stages_repeats: list,
                  stages_out_channels: list):
    img_input = layers.Input(shape=input_shape)
    if len(stages_repeats) != 3:
        raise ValueError("expected stages_repeats as list of 3 positive ints")
    if len(stages_out_channels) != 5:
        raise ValueError("expected stages_out_channels as list of 5 positive ints")

    x = ConvBNReLU(filters = stages_out_channels[0],
                   kernel_size=3,
                   strides=2,
                   name="conv1")(img_input)

    x = layers.MaxPooling2D(pool_size=(3, 3),
                            strides=2,
                            padding='same',
                            name="maxpool")(x)

    stage_name = ["stage{}".format(i) for i in [2, 3, 4]]
    for name, repeats, output_channels in zip(stage_name,
                                              stages_repeats,
                                              stages_out_channels[1:]):
        for i in range(repeats):
            if i == 0:
                x = shuffle_block_s2(x, output_c=output_channels, stride=2, prefix=name + "_{}".format(i))
            else:
                x = shuffle_block_s1(x, output_c=output_channels, stride=1, prefix=name + "_{}".format(i))

    x = ConvBNReLU(filters=stages_out_channels[-1], name="conv5")(x)

    x = layers.GlobalAveragePooling2D(name="globalpool")(x)

    x = layers.Dense(units=num_classes, name="fc")(x)
    x = layers.Softmax()(x)

    model = Model(img_input, x, name="ShuffleNetV2_1.0")

    return model


def shufflenet_v2_x1_0(num_classes=1000, input_shape=(224, 224, 3)):
    # 权重链接: https://pan.baidu.com/s/1M2mp98Si9eT9qT436DcdOw  密码: mhts
    model = shufflenet_v2(num_classes=num_classes,
                          input_shape=input_shape,
                          stages_repeats=[4, 8, 4],
                          stages_out_channels=[24, 116, 232, 464, 1024])
    return model


def shufflenet_v2_x0_5(num_classes=1000, input_shape=(224, 224, 3)):
    model = shufflenet_v2(num_classes=num_classes,
                          input_shape=input_shape,
                          stages_repeats=[4, 8, 4],
                          stages_out_channels=[24, 48, 96, 192, 1024])
    return model


def shufflenet_v2_x2_0(num_classes=1000, input_shape=(224, 224, 3)):
    model = shufflenet_v2(num_classes=num_classes,
                          input_shape=input_shape,
                          stages_repeats=[4, 8, 4],
                          stages_out_channels=[24, 244, 488, 976, 2048])
    return model

model = shufflenet_v2_x1_0(input_shape = (im_height, im_width, 3), num_classes=num_classes)
model.summary()

In [ ]:
pre_weights_path = './shufflenetv2_x1_0.h5'
assert os.path.exists(pre_weights_path), "cannot find {}".format(pre_weights_path)
model.load_weights(pre_weights_path, by_name=True, skip_mismatch=True)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name = 'train_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy(name='val_accuracy')

In [ ]:
best_val_acc = 0.

# custom learning rate curve
def scheduler(now_epoch):
    initial_lr = 0.1
    end_lr_rate = 0.1  # end_lr = initial_lr * end_lr_rate
    rate = ((1 + math.cos(now_epoch * math.pi / epochs)) / 2) * (1 - end_lr_rate) + end_lr_rate

    # cosine
    new_lr = rate * initial_lr

    # writing lr into tensorboard
    with train_writer.as_default():
        tf.summary.scalar('learning rate', data=new_lr, step=epoch)

    return new_lr

for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()

    train_bar = tqdm(train_dataset, file=sys.stdout)
    print(train_bar)
    for images, labels in train_bar:
        #train_step(images, lables)
        with tf.GradientTape() as tape:
            output = model(images, training=True)
            loss = loss_object(labels, output)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, output)

        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}, acc:{:.3f}".format(epoch + 1,
                                                                             epochs,
                                                                             train_loss.result(),
                                                                             train_accuracy.result())

    optimizer.learning_rate = scheduler(epoch)

    val_bar = tqdm(val_dataset, file=sys.stdout)
    for images, labels in val_bar:
        output = model(images, training=False)
        loss = loss_object(labels, output)

        val_loss(loss)
        val_accuracy(labels, output)

        val_bar.desc = "val epoch[{}/{}] loss:{：3f} acc:{:.3f}".format(epoch + 1,
                                                                       epochs,
                                                                       val_loss.result(),
                                                                       val_accuracy().result())


    # writing training loss and acc
    with train_writer.as_default():
        tf.summary.scalar("loss", train_loss.result(), epoch)
        tf.summary.scalar("accuracy", train_accuracy.result(), epoch)

    # writing validation loss and acc
    with val_writer.as_default():
        tf.summary.scalar("loss", val_loss.result(), epoch)
        tf.summary.scalar("accuracy", val_accuracy.result(), epoch)

    # only save best weights
    if val_accuracy.result() > best_val_acc:
        best_val_acc = val_accuracy.result()
        model.save_weights("./save_weights/shufflenetv2.ckpt", save_format="tf")



In [ ]:
import os
import json
import glob
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt



def main():
    im_height = 224
    im_width = 224
    num_classes = 5

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # load image
    img_path = "../tulip.jpg"
    assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
    img = Image.open(img_path)
    # resize image to 224x224
    img = img.resize((im_width, im_height))
    plt.imshow(img)

    # scaling pixel value to (-1,1)
    img = np.array(img).astype(np.float32)
    img = (img / 255. - mean) / std

    # Add the image to a batch where it's the only member.
    img = (np.expand_dims(img, 0))

    # read class_indict
    json_path = './class_indices.json'
    assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

    with open(json_path, "r") as f:
        class_indict = json.load(f)

    # create model
    model = shufflenet_v2_x1_0(num_classes=num_classes)

    weights_path = './save_weights/shufflenetv2.ckpt'
    assert len(glob.glob(weights_path+"*")), "cannot find {}".format(weights_path)
    model.load_weights(weights_path)

    result = np.squeeze(model.predict(img))
    predict_class = np.argmax(result)

    print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_class)],
                                                 result[predict_class])
    plt.title(print_res)
    for i in range(len(result)):
        print("class: {:10}   prob: {:.3}".format(class_indict[str(i)],
                                                  result[i]))
    plt.show()


if __name__ == '__main__':
    main()